In [2]:
!pip install fake_useragent

  Stored in directory: C:\Users\Mi\AppData\Local\pip\Cache\wheels\5e\63\09\d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\Mi\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _dep_map
    return self.__dep_map
  File "C:\Users\Mi\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Mi\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3003, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\Mi\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Mi\Anaconda3\lib\site-packages\pip\_internal\commands\in

In [6]:
import requests
import re
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [8]:
ua = UserAgent(verify_ssl=False)
session = requests.session()

url = f'https://stihi.ru'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [43]:
topref = re.search(r'<a href="(.*?)" class="topref">Рекомендуемые произведения', page).groups()[0]
req = session.get(url + topref, headers={'User-Agent': ua.random})
html = req.text

In [44]:
def get_info(html):
    soup = BeautifulSoup(html,'html.parser')
    poems = [x.get_text() for x in soup.find_all('a', {'class': 'poemlink'})]
    authors = [x.get_text() for x in soup.find_all('a', {'class': 'authorlink'})]
    dates = [re.search(r'(\d\d\.\d\d\.\d{4})', x.get_text()).groups()[0] for x in soup.find_all('small')]
    return poems,authors,dates

In [48]:
soup = BeautifulSoup(html,'html.parser')

In [49]:
recommendations = []
for a in soup.find('div',{'class':'textlink nounline'}).find_all('a'):
    req = session.get(url + a.get('href'), headers={'User-Agent': ua.random})
    html = req.text
    poems, authors, dates = get_info(html)
    recommendations.extend(zip(poems,authors,dates))

Сортировать по времени удобно в пандас сразу

In [51]:
import pandas as pd

In [56]:
rec = pd.DataFrame(recommendations, columns=['poem','author','date'])
rec.date = pd.to_datetime(rec.date, dayfirst=True)

In [70]:
rec.sort_values('date').head()

,poem,author,date
19,Ярмарка,Ольга Суханова,2017-11-12
187,Движения всегда быстры,Сергей Низамов,2020-07-17
192,Какою-то сказочной Русью,Сергей Хомутов Рыбинск,2020-07-17
190,Здесь все поддерживают форму,Мария Знобищева,2020-07-17
189,Губ твоих немая пристань,Татьяна Янчукович,2020-07-17


In [68]:
for poem, author, date in rec.sort_values('date').values:
    print(author, 'выложил(-а) стихотворение "' + poem + '"', date.date())

Ольга Суханова выложил(-а) стихотворение "Ярмарка" 2017-11-12
Сергей Низамов выложил(-а) стихотворение "Движения всегда быстры" 2020-07-17
Сергей Хомутов Рыбинск выложил(-а) стихотворение "Какою-то сказочной Русью" 2020-07-17
Мария Знобищева выложил(-а) стихотворение "Здесь все поддерживают форму" 2020-07-17
Татьяна Янчукович выложил(-а) стихотворение "Губ твоих немая пристань" 2020-07-17
Сергей Шелковый выложил(-а) стихотворение "Зеленая слива становится белой" 2020-07-17
Марианна Боровкова выложил(-а) стихотворение "Где цветут петуньи" 2020-07-17
Владимир Шугля выложил(-а) стихотворение "В родимом краю" 2020-07-17
Владимир Мялин выложил(-а) стихотворение "Я был актёр, но не бродячий" 2020-07-18
Елена Викман выложил(-а) стихотворение "Ягодные феи" 2020-07-18
Евгений Глушаков выложил(-а) стихотворение "Я думал - хранимая бережно ваза" 2020-07-19
Нина Дедлова выложил(-а) стихотворение "Нежданно-негаданно  выплыли строки" 2020-07-29
Лина Маго выложил(-а) стихотворение "Осеннее брожение" 